In [ ]:
############################################ LIBRAIRIES #####################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import xmltodict
# from scipy.signal import resample
import os
import h5py
import csv

import tensorflow as tf
import tensorflow_transform as tft
from tfx import v1 as tfx

import tensorflow_data_validation as tfdv
from tfx.components import CsvExampleGen, ImportExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator
from tfx.proto import example_gen_pb2, transform_pb2, trainer_pb2, evaluator_pb2
from tfx.utils import io_utils
#from tfx.types import channel

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from sklearn.model_selection import train_test_split
from tensorflow_metadata.proto.v0 import schema_pb2
from EcgPreprocessing import ECGDataset
from Labels import XMLtoCSV
import hdf5_to_TFRecords

######################################## CHEMINS ################################################


ECG_DIR = './xml_data/xml_ecg' # ECG sous format xml
ECG_OUTPUT = './xml_data/dataset/FinalTracings.h5' # ECG sous format hdf5
ECG_TRACINGS = './xml_data/dataset/FinalTracings.tfrecord' # ECG sous format TFRecord
LABELS_OUTPUT_FILE = './xml_data/labels_ecg/finals_labels.csv'# Labels en csv


PIPELINE_ROOT = './xml_data/Pipelines' # Dossiers contentant les artéfacts des composants
TRAIN_DIR = 'xml_data/dataset/splits/train' # Données d'entrainement hdf5
TEST_DIR = 'xml_data/dataset/splits/test' # Données de test hdf5

TRAIN_TRACINGS = 'xml_data/dataset/records_splits/train/' 
TEST_TRACINGS = 'xml_data/dataset/records_splits/test/'


 ## Créons un contexte Interactif
 Lors de la mise en production, nous souhaitons automatiser l'exécution du pipeline en utilisant des orchestrateurs tels que Apache Beam ou Kubeflow. On ne le fera pas tout de suite car nous exécuterons le pipeline à partir de ce notebook. Lorsque nous expérimenterons dans un environnement de notebook(jupyter), nous serons amener à exécuterez manuellement les composants du pipeline (c'est-à-dire que nous serons l'orchestrateur). 
 Pour cela, TFX fournit le **Contexte interactif** : `InteractiveContext` afin que nous puissions parcourir chaque composant et inspecter ses résultats.

In [2]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Prétraitement & Étiquetage

In [3]:
# Prétraitement
ecg_dataset = ECGDataset(input_dir=ECG_DIR, output_file=ECG_OUTPUT)
ecg_dataset.xml_to_tensor()

# Étiquetage
xml_to_csv = XMLtoCSV(folder_name=ECG_DIR)
xml_to_csv.convert(output_file=LABELS_OUTPUT_FILE)

(12, 4096, 12)


## Séparation des Données : Train - Test / HDF5 -> TFRecord

In [4]:
#!python hdf5_to_TFRecords.py --input-file-path ./xml_data/dataset --output-file-path ./xml_data/dataset/ -r

In [5]:
"""
    Convertit les données d'un fichier HDF5 en un fichier CSV cohérent
    avec le format HDF5.

    :param input_path: Le chemin d'accès vers le fichier HDF5.
    :param output_path: Le chemin d'accès vers le fichier CSV de sortie.
"""
# def convert_hdf5_to_csv(input_path, output_path):
#     # Charger les données HDF5
#     with h5py.File(input_path, 'r') as hdf:
#         data = hdf['data'][:]

#     # Transposer les données pour que les dérivations soient les colonnes et les échantillons les lignes
#     data_transposed = np.transpose(data, (0, 2, 1))

#     # Créer un tableau de noms de colonnes
#     column_names = []
#     for i in range(12):
#         for j in range(4096):
#             column_names.append(f"derivation_{i}_echantillon_{j}")

#     # Créer un DataFrame pandas à partir des données transposées et des noms de colonnes
#     df = pd.DataFrame(data_transposed.reshape(data_transposed.shape[0], -1), columns=column_names)

#     # Exporter les données en CSV
#     df.to_csv(output_path, index=False)



""" 
    Cette fonction load_data permet de charger les données depuis un fichier HDF5 et les étiquettes 
    (labels) depuis un fichier CSV, puis de les diviser en données d'entraînement et données de test
"""
    
def load_data(hdf5_file, csv_file):
    # Chargement des données HDF5
    with h5py.File(hdf5_file, 'r') as hf:
        data = hf['tracings'][:]

    # Chargement des étiquettes depuis le fichier CSV
    labels = pd.read_csv(csv_file)

    # Diviser les données en données d'entraînement et en données de test
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    print("========================================================")
    print(len(X_train),len( X_test), len(y_train),len( y_test))

    return X_train, X_test, y_train, y_test


In [6]:
X_train, X_test, y_train, y_test = load_data(ECG_OUTPUT, LABELS_OUTPUT_FILE)

# Écrire les données dans les fichiers HDF5
with h5py.File(os.path.join(TRAIN_DIR, 'X_train.h5'), 'w') as f:
    f.create_dataset('tracings', data=X_train)
with h5py.File(os.path.join(TEST_DIR, 'X_test.h5'), 'w') as f:
    f.create_dataset('tracings', data=X_test)

# ecg_records = convert_hdf5_to_csv(input_path=ecg_output, output_path=ecg_tracings) 
# X_train_records = convert_hdf5_to_csv(os.path.join(train_dir, 'X_train.hdf5'), output_path=os.path.join(train_tracings, "X_train.csv"))
# X_test_records =  convert_hdf5_to_csv(os.path.join(test_dir, 'X_test.hdf5'), output_path=os.path.join(test_tracings, "X_test.csv"))

# Écrire les labels dans les fichiers CSV
pd.DataFrame(y_train).to_csv(os.path.join(TRAIN_DIR, 'y_train.csv'), index=False)
pd.DataFrame(y_test).to_csv(os.path.join(TEST_DIR, 'y_test.csv'), index=False)

(9, 4096, 12) (3, 4096, 12) (9, 1) (3, 1)
9 3 9 3


In [ ]:
#!python hdf5_to_TFRecords.py --input-file-path ./xml_data/dataset/splits/train --output-file-path ./xml_data/dataset/splits/train -r

In [ ]:
#!python hdf5_to_TFRecords.py --input-file-path ./xml_data/dataset/splits/test --output-file-path ./xml_data/dataset/splits/test -r

In [7]:

# ExampleGen

"""
Tout d'abord, on définit la configuration d'entrée pour le composant ecg_gen en utilisant 
le protocole example_gen_pb2.Input. La configuration d'entrée contient une liste de splits, 
qui spécifient les répertoires d'entrée à utiliser pour chaque fraction de données. Dans cet exemple, 
il y a deux fractions: "train" et "eval", qui sont des répertoires situés dans le répertoire splits.

Ensuite on exécute le composant ecg_gen en appelant la méthode run du contexte TFX (context) avec 
le composant ecg_gen en argument. Cela déclenche l'exécution du composant, qui importera les données 
d'entraînement et d'évaluation à partir des répertoires spécifiés et générera des exemples TFX à utiliser pour 
la formation et l'évaluation des modèles.


"""

# Configuration des entrées pour les générateurs d'exemples

ecg_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern=os.path.join("/home/charlie/Documents/af_prediction_pipelines/", TRAIN_TRACINGS, "X_train.tfrecord")),
    example_gen_pb2.Input.Split(name='eval', pattern=os.path.join("/home/charlie/Documents/af_prediction_pipelines/",TEST_TRACINGS, "X_test.tfrecord"))
])

labels_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern=os.path.join("/home/charlie/Documents/af_prediction_pipelines/", TRAIN_DIR, "y_train.csv")),
    example_gen_pb2.Input.Split(name='eval', pattern=os.path.join("/home/charlie/Documents/af_prediction_pipelines/", TEST_DIR, "y_test.csv"))
])

""" 
Deux générateurs d'exemples sont créés en utilisant les configurations d'entrée précédemment définies. 
Le premier générateur d'exemples, ecg_gen, est un ImportExampleGen qui est utilisé pour importer les données de l'ECG 
à partir du répertoire ecg_dir. Le deuxième générateur d'exemples, labels_gen, est un CsvExampleGen qui est utilisé 
pour charger les étiquettes à partir du fichier CSV de labels_output_file.

La méthode generate_fingerprint de io_utils est utilisée pour générer une empreinte unique pour le fichier CSV 
afin de garantir que les données d'entrée pour chaque exécution du pipeline sont les mêmes.
Cela est important pour assurer la reproductibilité et la cohérence des résultats.

"""
ecg_gen = ImportExampleGen(input_base=ECG_TRACINGS, input_config=ecg_config)

labels_gen = CsvExampleGen(input_base=io_utils.generate_fingerprint(split_name="labels", file_pattern=LABELS_OUTPUT_FILE), input_config=labels_config)


In [8]:
context.run(labels_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 74
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [9]:
context.run(ecg_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 75
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Statistiques de notre ensemble de données

In [24]:
ecg_statistics_gen = tfx.components.StatisticsGen(
    examples=ecg_gen.outputs['examples']
)

# Execute the component
context.run(ecg_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 81
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [25]:
context.show(ecg_statistics_gen.outputs['statistics'])

## SCHEMA de nos données avec SchemaGen

Le composant `SchemaGen` utilise également TFDV pour générer un schéma basé sur les statistiques de nos données. Un schéma définit les limites, les types et les propriétés attendus des caractéristiques de votre jeu de données.



In [12]:

ecg_schema_gen = tfx.components.SchemaGen(
    statistics=ecg_statistics_gen.outputs['statistics'],
    )

context.run(ecg_schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 77
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [13]:
# Visualize the schema
context.show(ecg_schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AVF',FLOAT,required,,-
'AVL',FLOAT,required,,-
'AVR',FLOAT,required,,-
'DI',FLOAT,required,,-
'DII',FLOAT,required,,-
'DIII',FLOAT,required,,-
'V1',FLOAT,required,,-
'V2',FLOAT,required,,-
'V3',FLOAT,required,,-


## ExampleValidator

Le composant `ExampleValidator` détecte les anomalies dans vos données sur la base du schéma généré à l'étape précédente. Comme les deux composants précédents, il utilise également `TFDV` sous le capot. Par défaut, il compare les statistiques de l'évaluation au schéma de l'entraînement.



In [19]:
ecg_example_validator = ExampleValidator(
    statistics=ecg_statistics_gen.outputs['statistics'],
    schema=ecg_schema_gen.outputs["schema"]
)
context.run(ecg_example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 79
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))